In [1]:
# !pip install langgraph langchain-google-genai python-dotenv faiss-cpu

In [2]:
import os
import pickle
from dotenv import load_dotenv

from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage

load_dotenv()

True

In [3]:
from typing import TypedDict, List, Dict, Any

class LifeAllyState(TypedDict):
    user_query: str
    domain: str
    model_outputs: Dict[str, Any]
    retrieved_texts: List[str]
    response: str
    profile: Dict[str, Any] 

In [4]:
def welcome_node(state: LifeAllyState) -> LifeAllyState:
    state["response"] = (
        "👋 **Welcome to LifeAlly – Your AI Life Coach!**\n\n"
        "I can help you in these areas:\n"
        " - 🧑‍💼 Career guidance\n"
        " - 💰 Finance advice\n"
        " - ❤️ Relationship support\n"
        " - 🏥 Health and wellness\n\n"
        "Please choose a domain or just type your situation!\n\n"
        "**Sample queries:**\n"
        " - Career: _'I'm confused about which career path to choose.'_\n"
        " - Finance: _'How can I save more money each month?'_\n"
        " - Health: _'How to improve my sleep?'_\n"
        " - Relationship: _'How do I handle conflicts with my partner?'_\n\n"
        "Type a domain (Career, Finance, Health, Relationship) or describe your problem to begin."
    )
    return state

In [5]:
import joblib
career_model_1=joblib.load("../models/career_admission_model.pkl")
career_model_2=joblib.load("../models/career_growth_model.pkl")
career_model_3=joblib.load("../models/career_profession_model.pkl")
finance_model_1=joblib.load("../models/finance_disposable_income_model.pkl")
finance_model_2=joblib.load("../models/finance_loan_model.pkl")
finance_model_3=joblib.load("../models/finance_personal_tracker_model.pkl")
health_model_1=joblib.load("../models/mental_health_model.pkl")
health_model_2=joblib.load("../models/heart_health_xgb_model.pkl")
relationship_model_1=joblib.load("../models/relationship_emotion_model.pkl")
relationship_model_2=joblib.load("../models/relationship_model.pkl")
relationship_model_3=joblib.load("../models/relationship_tfidf_knn.pkl")

print("All models are successfully loaded!")

All models are successfully loaded!


In [6]:
def extract_structured_features(user_query: str, expected_features: int):
    """
    Extracts comma-separated features if the count matches expected_features.
    """
    features = [x.strip() for x in user_query.split(",")]
    if len(features) == expected_features:
        return features
    else:
        return None

In [7]:
FEATURE_NAMES = {
    "career_admission": [
        "Age", "Gender", "Education Level", "Field of Interest", "Relevant Experience", "Preferred Location", "Expected Salary"
    ],
    "career_growth": [
        # List the 17 features in order as expected by the career_growth model
        # Example (fill in with your actual order):
        "Age", "Gender", "Education Level", "Current Role", "Years of Experience", "Field", "Industry", "Current Salary",
        "Desired Role", "Skills", "Certifications", "Location", "Mentorship", "Company Size", "Work-Life Balance", "Career Goals", "Motivation Level"
    ],
    "career_profession": [
        # List the 18 features in order as expected by the career_profession model
        # Example (fill in with your actual order):
        "Age", "Gender", "Education", "Field", "Current Role", "Experience", "Skills", "Certs", "Industry", "Location",
        "Salary", "Job Type", "Company Size", "Work Preference", "Motivation", "Personality", "Growth Mindset", "Ambition"
    ],
    "finance_income": [
        # List the 18 features in order as expected by the finance_income model
        # Example (fill in with your actual order):
        "Annual Income", "Age", "Dependents", "Career Level", "Industry", "Rent/Mortgage", "Groceries", "Transportation",
        "Savings", "Discretionary Spending", "Utilities", "Loan Payment", "Insurance", "Entertainment", "Education", "Miscellaneous", "Years to Retirement", "Desired Savings"
    ],
    "finance_loan": [
        # List the 17 features in order as expected by the finance_loan model
        # Example (fill in with your actual order):
        "Age", "Gender", "Education", "Employment", "Profession", "Income", "Expenses", "Own House", "Loan Type",
        "Loan Amount", "Loan Tenure (months)", "Loan EMI", "Interest Rate", "Credit Utilization", "Credit Score", "Savings Ratio", "Country"
    ],
    "finance_tracker": [
        # List the 21 features in order as expected by the finance_tracker model
        # Example (fill in with your actual order):
        "Income", "Expenses", "Savings Ratio", "Debt Amount", "Status", "Credit Score", "Debt Ratio", "EMI", "Net Worth", "Investments",
        "Insurance", "Goals", "Missed Payments", "Retirement Fund", "Annual Savings", "Primary Income Source", "Secondary Income Source", "Financial Health", "Spending Score", "Risk Appetite", "Liquidity"
    ],
    "health_mental": [
        "Country", "Age", "Gender", "Exercise Level", "Diet Type", "Sleep Hours", "Stress Level", "Mental Health Condition",
        "Work Hours per Week", "Screen Time per Day (Hours)", "Social Interaction Score", "Happiness Score"
    ],
    "health_heart": [
        "Age", "Gender", "Region", "Urban/Rural", "SES", "Smoking Status", "Alcohol Consumption", "Diet Type", "Physical Activity Level",
        "Screen Time (hrs/day)", "Family History of Heart Disease", "Hypertension", "Diabetes", "Cholesterol Levels", "BMI", "Sleep Duration",
        "Stress Level", "Blood Pressure (systolic/diastolic mmHg)", "Resting Heart Rate (bpm)", "ECG Results", "Chest Pain Type",
        "Maximum Heart Rate Achieved", "Exercise Induced Angina", "Blood Oxygen Levels (SpO2%)", "Triglyceride Levels (mg/dL)"
    ]
}

In [8]:
# LLM-based natural language to structured feature extraction node

def make_llm_extract_node(model_name):
    """
    Returns a node function that uses Gemini LLM to extract structured features in correct order from natural language.
    """
    feature_names = FEATURE_NAMES[model_name]
    def node_fn(state: LifeAllyState) -> LifeAllyState:
        user_query = state["user_query"]
        # Prompt Gemini to extract the values in order
        prompt = (
            f"You are an expert life coach AI assistant. Extract the following details from the user's message, "
            f"and output a comma-separated list in this exact order (only the values, no labels):\n"
            f"{', '.join(feature_names)}\n\n"
            f"User message:\n{user_query}\n\n"
            f"If a value is missing or unclear, output a question mark (?) for that value. Use no extra text."
        )
        llm_output = llm.invoke([HumanMessage(content=prompt)]).content.strip()
        # Save the extracted features as the user_query for the model node
        state["user_query"] = llm_output
        # Optionally, store the extraction for later debugging:
        state["model_outputs"][f"{model_name}_extracted"] = llm_output
        return state
    return node_fn

In [9]:
def get_feature_list_for_model(profile, feature_names):
    """Return values in the order required by the model, using '?' for missing."""
    return [profile.get(name, "?") for name in feature_names]

def make_model_node(model_name, model, expected_features=None):
    """
    Wraps a model into a LangGraph node. Handles errors gracefully and prompts for more info if needed.
    Uses profile memory for multi-turn support!
    """
    def node_fn(state: LifeAllyState) -> LifeAllyState:
        # Use profile dict if available, else fallback to user_query
        feature_names = FEATURE_NAMES.get(model_name, [])
        profile = state.get("profile", {})
        features = None

        if expected_features and profile and feature_names:
            features = get_feature_list_for_model(profile, feature_names)

            if features.count("?") > 0 or len(features) != expected_features:
                features = None  

        if features is None and expected_features:
            user_input = state["user_query"]
            features = extract_structured_features(user_input, expected_features)

        try:
            if expected_features and (features is None or features.count("?") > 0):

                feature_list_str = "\n".join(
                    [f"{i+1}. {name}" for i, name in enumerate(feature_names)]
                ) if feature_names else "<feature names not available>"
                prediction = (
                    f"To provide tailored advice using the '{model_name.replace('_', ' ').title()}' model, "
                    f"I need {expected_features} specific details (comma-separated, in the following order):\n"
                    f"{feature_list_str}\n"
                    "Please provide these as a comma-separated list (in the order above)."
                )
            else:
                input_for_model = features if features is not None else [state["user_query"]]
                prediction = model.predict([input_for_model])[0]
        except Exception as e:
            prediction = "Sorry, there was an internal error running this model."
        state["model_outputs"][model_name] = prediction
        return state

    return node_fn

career_admission_node = make_model_node("career_admission", career_model_1, expected_features=7)
career_growth_node    = make_model_node("career_growth", career_model_2, expected_features=17)
career_profession_node= make_model_node("career_profession", career_model_3, expected_features=18)

finance_income_node   = make_model_node("finance_income", finance_model_1, expected_features=18)
finance_loan_node     = make_model_node("finance_loan", finance_model_2, expected_features=17)
finance_tracker_node  = make_model_node("finance_tracker", finance_model_3, expected_features=21)

health_mental_node    = make_model_node("mental_health", health_model_1, expected_features=12)
health_heart_node     = make_model_node("heart_health", health_model_2, expected_features=25)

relationship_emotion_node = make_model_node("relationship_emotion", relationship_model_1)
relationship_node         = make_model_node("relationship", relationship_model_2)
relationship_knn_node     = make_model_node("relationship_knn", relationship_model_3)

In [10]:
def make_profile_update_node(feature_names):
    def node_fn(state: LifeAllyState) -> LifeAllyState:
        user_query = state["user_query"]
        profile = state.get("profile", {})

        prompt = (
            f"Extract any of these fields and their values from the user's message:\n{', '.join(feature_names)}.\n"
            f"Output as: Field: Value, Field: Value, ... Only output fields you can extract. "
            f"If none, output nothing."
            f"\nUser message: {user_query}"
        )
        llm_output = llm.invoke([HumanMessage(content=prompt)]).content.strip()
        updates = {}
        for item in llm_output.split(","):
            if ":" in item:
                key, value = [x.strip() for x in item.split(":", 1)]
                if key in feature_names and value not in ["?", ""]:
                    updates[key] = value
        profile.update(updates)
        state["profile"] = profile
        return state
    return node_fn

In [11]:
career_admission_extract = make_llm_extract_node("career_admission")
career_growth_extract = make_llm_extract_node("career_growth")
career_profession_extract = make_llm_extract_node("career_profession")
finance_income_extract = make_llm_extract_node("finance_income")
finance_loan_extract = make_llm_extract_node("finance_loan")
finance_tracker_extract = make_llm_extract_node("finance_tracker")
health_mental_extract = make_llm_extract_node("health_mental")
health_heart_extract = make_llm_extract_node("health_heart")

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GEMINI_API_KEY,
    temperature=0.3
)

def gemini_node(state: LifeAllyState) -> LifeAllyState:
    user_query = state["user_query"]
    model_outputs = state["model_outputs"]
    retrieved_texts = state.get("retrieved_texts", [])
    filtered_outputs = {
        k: v for k, v in model_outputs.items()
        if not v.lower().startswith("to provide tailored advice") and not v.lower().startswith("sorry,")
    }
    missing_data = any(
        v.lower().startswith("to provide tailored advice") for v in model_outputs.values()
    )
    prompt = f"""
You are a professional AI life coach. Respond to the user in a supportive, clear, and actionable way, relevant emojis to make the advice more engaging and approachable.

User asked: {user_query}

Model insights:
{filtered_outputs if filtered_outputs else "No specific insights due to missing details."}

Background info:
{retrieved_texts}

{"NOTE: The system needs more details for personalized advice." if missing_data else ""}

Provide a concise, motivational summary and concrete next steps for the user.
"""
    response = llm.invoke([HumanMessage(content=prompt)])
    state["response"] = response.content
    return state

In [13]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(LifeAllyState)

# Add welcome node, set as entry point
workflow.add_node("welcome", welcome_node)
workflow.set_entry_point("welcome")
workflow.add_edge("welcome", "career_admission_profile_update")

# --- Add PROFILE UPDATE nodes for each pipeline ---
workflow.add_node("career_admission_profile_update", make_profile_update_node(FEATURE_NAMES["career_admission"]))
workflow.add_node("career_growth_profile_update", make_profile_update_node(FEATURE_NAMES["career_growth"]))
workflow.add_node("career_profession_profile_update", make_profile_update_node(FEATURE_NAMES["career_profession"]))
workflow.add_node("finance_income_profile_update", make_profile_update_node(FEATURE_NAMES["finance_income"]))
workflow.add_node("finance_loan_profile_update", make_profile_update_node(FEATURE_NAMES["finance_loan"]))
workflow.add_node("finance_tracker_profile_update", make_profile_update_node(FEATURE_NAMES["finance_tracker"]))
workflow.add_node("health_mental_profile_update", make_profile_update_node(FEATURE_NAMES["health_mental"]))
workflow.add_node("health_heart_profile_update", make_profile_update_node(FEATURE_NAMES["health_heart"]))

# --- Add extract and model nodes for career ---
workflow.add_node("career_admission_extract", career_admission_extract)
workflow.add_node("career_admission", career_admission_node)
workflow.add_node("career_growth_extract", career_growth_extract)
workflow.add_node("career_growth", career_growth_node)
workflow.add_node("career_profession_extract", career_profession_extract)
workflow.add_node("career_profession", career_profession_node)

# --- Add extract and model nodes for finance ---
workflow.add_node("finance_income_extract", finance_income_extract)
workflow.add_node("finance_income", finance_income_node)
workflow.add_node("finance_loan_extract", finance_loan_extract)
workflow.add_node("finance_loan", finance_loan_node)
workflow.add_node("finance_tracker_extract", finance_tracker_extract)
workflow.add_node("finance_tracker", finance_tracker_node)

# --- Add extract and model nodes for health ---
workflow.add_node("health_mental_extract", health_mental_extract)
workflow.add_node("health_mental", health_mental_node)
workflow.add_node("health_heart_extract", health_heart_extract)
workflow.add_node("health_heart", health_heart_node)

# --- Relationship nodes (no extract node needed) ---
workflow.add_node("relationship_emotion", relationship_emotion_node)
workflow.add_node("relationship_classifier", relationship_node)
workflow.add_node("relationship_knn", relationship_knn_node)

# --- Gemini node ---
workflow.add_node("gemini", gemini_node)

# --- Edges for career pipeline ---
workflow.add_edge("career_admission_profile_update", "career_admission_extract")
workflow.add_edge("career_admission_extract", "career_admission")
workflow.add_edge("career_admission", "career_growth_profile_update")
workflow.add_edge("career_growth_profile_update", "career_growth_extract")
workflow.add_edge("career_growth_extract", "career_growth")
workflow.add_edge("career_growth", "career_profession_profile_update")
workflow.add_edge("career_profession_profile_update", "career_profession_extract")
workflow.add_edge("career_profession_extract", "career_profession")
workflow.add_edge("career_profession", "gemini")

# --- Edges for finance pipeline ---
workflow.add_edge("finance_income_profile_update", "finance_income_extract")
workflow.add_edge("finance_income_extract", "finance_income")
workflow.add_edge("finance_income", "finance_loan_profile_update")
workflow.add_edge("finance_loan_profile_update", "finance_loan_extract")
workflow.add_edge("finance_loan_extract", "finance_loan")
workflow.add_edge("finance_loan", "finance_tracker_profile_update")
workflow.add_edge("finance_tracker_profile_update", "finance_tracker_extract")
workflow.add_edge("finance_tracker_extract", "finance_tracker")
workflow.add_edge("finance_tracker", "gemini")

# --- Edges for health pipeline ---
workflow.add_edge("health_mental_profile_update", "health_mental_extract")
workflow.add_edge("health_mental_extract", "health_mental")
workflow.add_edge("health_mental", "health_heart_profile_update")
workflow.add_edge("health_heart_profile_update", "health_heart_extract")
workflow.add_edge("health_heart_extract", "health_heart")
workflow.add_edge("health_heart", "gemini")

# --- Edges for relationship pipeline ---
workflow.add_edge("relationship_emotion", "relationship_classifier")
workflow.add_edge("relationship_classifier", "relationship_knn")
workflow.add_edge("relationship_knn", "gemini")

# --- Complete the workflow ---
workflow.add_edge("gemini", END)

app = workflow.compile()

In [14]:
initial_state = {
    "user_query": "I am confused about which career path to choose after graduation.",
    "domain": "career",
    "model_outputs": {},
    "retrieved_texts": [],
    "response": "",
    "profile": {}
}
final_state = app.invoke(initial_state)

In [15]:
from IPython.display import display, Markdown
import pandas as pd

# --- MAIN OUTPUT (AI Advice) ---
display(Markdown(final_state["response"]))

# --- MODEL OUTPUTS TABLE ---
model_outputs = final_state.get("model_outputs", {})
error_msgs = []

if model_outputs:
    filtered_outputs = {
        k: v for k, v in model_outputs.items()
        if v and not str(v).lower().startswith("to provide tailored advice")
    }
    if filtered_outputs:
        df = pd.DataFrame(list(filtered_outputs.items()), columns=["Model", "Output"])
        display(Markdown("### Model Outputs"))
        display(df)
        # Check for missing data or errors
        for model, output in filtered_outputs.items():
            val = str(output).strip().lower()
            if val in ["?", "??", "???", "none", "missing", ""]:
                error_msgs.append(
                    f"- ⚠️ **{model}** did not receive enough information to make a prediction. Please provide all required details."
                )
            elif val.startswith("sorry, there was an internal error"):
                error_msgs.append(
                    f"- ❌ There was an **internal error** running **{model}**. Please try again later or contact support."
                )

# --- GRANULAR ERROR FEEDBACK FOR MAIN RESPONSE ---
main_response = final_state.get("response", "")
if ("question mark" in main_response.lower() or
    "need more details" in main_response.lower() or
    "missing details" in main_response.lower()):
    error_msgs.append(
        "⚠️ **Some important details were missing from your input.**\n\n"
        "For the most accurate advice, please provide all requested information."
    )

# --- PROFESSIONAL NEXT STEPS OR SUCCESS MESSAGE ---
if error_msgs:
    msg = "\n\n".join(error_msgs)
    display(Markdown(
        f"""---
**Feedback & Next Steps:**

{msg}

---
"""))
else:
    display(Markdown(
        "<span style='color:green; font-weight:bold'>✅ All required data present. You received a personalized response!</span>"
    ))

Hey there! 👋  I see you've got a lot on your mind regarding your career.  It's great that you're reaching out!  Unfortunately, those question marks don't give me much to work with. 🤔 To help you best, I need some specifics.

**Let's clarify your goals.**  Think about what's causing you uncertainty.  Is it:

* **Career Admission?**  (e.g., applying for a job, getting into a program)  Tell me more about the application process or program you're interested in.
* **Career Growth?** (e.g., promotion, new skills, changing roles) What are your aspirations? What steps have you already taken?
* **Career Profession?** (e.g., choosing a career path, switching careers) What are your interests and skills? What kind of work environment are you looking for?


**Next Steps:**

1. **Choose ONE area to focus on:**  Career admission, growth, or profession.  This will help us break things down. 🎯
2. **Write down 3 specific questions:**  Instead of question marks, give me concrete questions about your chosen area.  For example, instead of "?", write "How can I improve my resume for this specific job?" or "What skills do I need to develop for a promotion?".
3. **Share your answers:** Once you've done that, we can work together to create a personalized action plan! 💪


I'm here to support you every step of the way. Let's get started! ✨

### Model Outputs

,Model,Output
0,career_admission_extracted,"?, ?, ?, ?, ?, ?, ?"
1,career_growth_extracted,"?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?"
2,career_profession_extracted,"?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?"


---
**Feedback & Next Steps:**

⚠️ **Some important details were missing from your input.**

For the most accurate advice, please provide all requested information.

---


In [16]:
finance_income_query = (
    "45000, 30, 1, Professional, Tier_1, 12000, 2000, 1500, 5000, 2000, 1000, 800, 700, 600, 500, 400, 10, 5000"
)

finance_loan_query = (
    "35, Male, Master, Employed, Engineer, 6000, 2500, Yes, Personal, 20000, 36, 600, 7.5, 0.4, 720, 0.25, Europe"
)

finance_tracker_query = (
    "5000, 3000, 0.2, 1000, normal, 700, 0.3, 500, 10000, 50, 2000, 12, 0, 600, 2400, Salary, Investments, Positive, 80, Low, 2000"
)

for query, domain in [
    (finance_income_query, "finance_income"),
    (finance_loan_query, "finance_loan"),
    (finance_tracker_query, "finance_tracker"),
]:
    print(f"\n--- Testing {domain} ---")
    initial_state = {
        "user_query": query,
        "domain": "finance",
        "model_outputs": {},
        "retrieved_texts": [],
        "response": "",
        "profile": {}
    }
    final_state = app.invoke(initial_state)
    display(Markdown(final_state["response"]))


--- Testing finance_income ---


Hello!  It's great you're taking this step towards self-improvement! 🎉

I see you've identified yourself as a 30-year-old professional at Tier 1.  That's fantastic!  However, to give you the best possible life coaching, I need a little more information. 🤔

To help me understand your goals and challenges, please tell me more about:

* **Your Profession:** What exactly do you do?  Knowing your field will help me tailor advice to your specific career path. 💼
* **Your Current Situation:** What are you happy with? What are your frustrations?  What are your biggest challenges right now? 🤔
* **Your Goals:** What do you want to achieve in your career and personal life?  Short-term and long-term goals would be helpful. 🚀
* **Your Aspirations:** Where do you see yourself in 5 years? 10 years? ✨

Once I have this information, I can provide you with a personalized plan to help you thrive.  Let's work together to unlock your full potential! 💪


--- Testing finance_loan ---


Hi there!  It's great you're taking this step towards self-improvement! 🎉

I see you're a 35-year-old male engineer with a Master's degree, employed in Europe and earning €20,000.  That's a solid foundation! 💪 However, to give you the best life coaching, I need some more information.  Think of me as your personal GPS – I need your destination to guide you! 🗺️

**Concise Summary:** You've achieved a lot, but let's unlock your full potential! We need to clarify your goals to create a personalized plan.

**Concrete Next Steps:**

1. **Define your goals:** What are you hoping to achieve in your career and personal life?  (e.g., higher salary, new skills, work-life balance, new relationship, travel more?) 🤔  Write down 3-5 key goals.
2. **Identify your challenges:** What obstacles are preventing you from reaching those goals? (e.g., lack of confidence, limited networking, specific skills gap?) 🚧
3. **Fill in the missing information:**  The question marks in your profile are crucial!  Tell me about your current job satisfaction, your relationship status, your hobbies, and your financial goals.  This will help me tailor advice to *you*. 📝

Once I have this information, we can create a powerful action plan together!  Let's work towards a brighter future! ✨


--- Testing finance_tracker ---


Hey there!  I see you've got a lot on your mind regarding your career 🤔.  It's great that you're reaching out!  To give you the best support, I need a little more information.  Those question marks aren't very helpful 😉.

Let's break this down.  Tell me more about what's causing you uncertainty.  For example:

* **What are your career goals?**  Where do you see yourself in 5 years? ➡️
* **What are your current skills and experiences?** What are you good at?  💪
* **What aspects of your current career situation are you unhappy with?** What's causing you stress? 😫
* **What kind of work environment are you looking for?**  Do you prefer teamwork or independent work? 🤝 or 🧑‍💻

Once I have a clearer picture of your situation, I can offer more specific and helpful advice.  Let's work together to create a plan for your success! ✨  I'm here to help you every step of the way. 😊

In [17]:
health_mental_query = (
    "India, 28, Male, Moderate, Vegetarian, 7, Low, None, 40, 3, 8.2, 7.5"
)

health_heart_query = (
    "30, Male, North, Urban, Middle, Never, Occasionally, Vegetarian, Moderate, 4, Yes, No, No, 180, 24.5, 7, Medium, 120/80, 72, Normal, Non-anginal, 180, No, 98.2, 140"
)

for query, domain in [
    (health_mental_query, "health_mental"),
    (health_heart_query, "health_heart"),
]:
    print(f"\n--- Testing {domain} ---")
    initial_state = {
        "user_query": query,
        "domain": "health",
        "model_outputs": {},
        "retrieved_texts": [],
        "response": "",
        "profile": {}
    }
    final_state = app.invoke(initial_state)
    display(Markdown(final_state["response"]))


--- Testing health_mental ---


Hey there!  It's great you're taking this step towards self-improvement! 🎉

I see you've provided some basic information, but to truly help you craft a fulfilling life plan, I need a bit more detail.  Think of me as your personal AI cheerleader – I'm here to support you, but I need some information to run with! 🏃‍♀️💨

**Concise Summary:**  You're at a great age for exploring possibilities!  Let's unlock your potential by filling in some blanks.

**Concrete Next Steps:**

1. **Career:** What are your interests? What kind of work makes you feel energized? 🤔 What are your skills and talents?  Even if you're unsure, jot down *anything* that comes to mind.
2. **Goals:** What are your short-term (next 6 months) and long-term (next 5 years) goals?  These could be career-related, personal, or anything else important to you.  🎯
3. **Values:** What's truly important to you in life? (e.g., family, creativity, financial security, helping others).  Understanding your values will guide your decisions. ✨
4. **Obstacles:** What's holding you back? Are there any challenges you're facing? Identifying obstacles is the first step to overcoming them. 💪

Once you provide me with this information, we can create a personalized plan to help you achieve your goals.  Let's work together to make this year your best yet! 👍


--- Testing health_heart ---


Hello there! 👋  I see you've reached out for some life coaching, but I need a little more information to help you effectively.  Your responses were a bit... mysterious! 😉

To give you the best possible guidance, could you please tell me more about what's on your mind?  Specifically, what areas of your life are you hoping to improve?  Are you looking for advice on your career, relationships, personal growth, or something else entirely? 🤔

For now, let's focus on taking small, actionable steps:

**1.  Self-Reflection:** 📝 Take 15 minutes today to jot down three things you'd like to change or improve in your life.  Be as specific as possible!

**2.  Identify One Goal:** 🌟 Choose ONE of those three things and make it your primary focus for the next week.  Break this goal down into smaller, manageable steps.

**3.  Schedule Time:** 🗓️  Block out time in your schedule specifically dedicated to working towards this goal.  Even 15-30 minutes a day can make a difference!

Once you provide me with more details, I can offer more tailored and effective advice.  Let's work together to achieve your goals! 💪

In [18]:
relationship_emotion_query = (
    "I've been feeling like my partner doesn't listen to me, and it makes me both sad and a bit frustrated. I want to fix things, but I don't know how to start."
)

relationship_scenario_query = (
    "SUBREDDIT: r/relationships\n\n"
    "TITLE: My partner seems distant lately\n\n"
    "POST: Over the past month, my partner has been spending more time at work and less time communicating with me. "
    "We barely have meaningful conversations anymore, and I'm starting to worry we're drifting apart. I love them and want to reconnect, but I'm not sure what steps to take. Any advice?\n\n"
    "TL;DR: Partner is distant, relationship feels strained, how can I reconnect?"
)

for query, domain in [
    (relationship_emotion_query, "relationship_emotion"),
    (relationship_scenario_query, "relationship"),
    (relationship_scenario_query, "relationship_knn"),
]:
    print(f"\n--- Testing {domain} ---")
    initial_state = {
        "user_query": query,
        "domain": "relationship",
        "model_outputs": {},
        "retrieved_texts": [],
        "response": "",
        "profile": {}
    }
    final_state = app.invoke(initial_state)
    display(Markdown(final_state["response"]))


--- Testing relationship_emotion ---


Hey there! 👋  I see you've got a lot on your mind regarding your career.  It's great that you're reaching out!  Unfortunately, those question marks don't give me much to work with. 🤔 To help you best, I need some specifics.

**Let's clarify your situation:** To give you the most effective guidance, please tell me more about what's causing you concern.  For example:

* **What aspects of your career are you questioning?** (e.g., job satisfaction, career path, work-life balance, salary) 🤔
* **What are your goals?** (e.g., promotion, new job, career change) 🚀
* **What are your strengths and weaknesses?** (This helps us identify opportunities and areas for improvement) 💪 Weaknesses
* **What resources do you have available?** (e.g., mentors, networks, training opportunities) 📚

Once I have a clearer picture, I can provide personalized advice and actionable steps.  Let's work together to create a plan for your success! ✨  Don't hesitate to share – even small details can make a big difference. 😊


--- Testing relationship ---


Hey there!  I see you've got a lot on your mind! 🤔  Those question marks tell me you're facing some big decisions, possibly related to your career.  It's great that you're reaching out! 👍

Unfortunately, I need a little more information to give you the best advice.  To help me understand your situation better, could you please tell me more about what's causing you to ask so many questions?  For example:

* **What area of your career are you struggling with?** (e.g., job search, career change, promotion, work-life balance) 💼
* **What are some specific questions you have?**  The more detail, the better! 📝
* **What are your goals?** Where do you see yourself in 1 year? 5 years? ✨

Once I have a clearer picture, I can help you create a personalized action plan.  Let's work together to turn those question marks into exclamation points! 🎉  I'm here to support you every step of the way. 😊


--- Testing relationship_knn ---


Hey there! 👋  I see you've got a lot on your mind regarding your career.  It's great that you're reaching out!  Unfortunately, those question marks don't give me much to work with. 🤔 To help you best, I need some specifics.

**Let's clarify your situation:**

To give you the most effective guidance, please tell me more about what's concerning you.  For example:

* **What aspects of your career are you questioning?**  (e.g., job satisfaction, career path, work-life balance, salary, skills development) 🤔
* **What are your career goals?** (short-term and long-term) 🚀
* **What are your strengths and weaknesses?**  Knowing this helps us identify opportunities and areas for improvement. 💪 Weaknesses
* **What resources do you have available?** (e.g., time, money, network) 💰

Once I have a clearer picture, I can provide personalized advice and actionable steps.  Let's work together to create a fulfilling and successful career journey! ✨

In [23]:
# Install psycopg2 if not already installed
# !pip install psycopg2-binary

import psycopg2

# Connect to your PostgreSQL DB (update password accordingly)
conn = psycopg2.connect(
    dbname="lifeally",
    user="postgres",
    password="Shelar",  # <-- put your actual password
    host="localhost",
    port="5432"
)

cur = conn.cursor()
cur.execute("SELECT version();")
print("PostgreSQL version:", cur.fetchone())

# Optional: Test creating a table and inserting data
cur.execute("CREATE TABLE IF NOT EXISTS test_table (id SERIAL PRIMARY KEY, name VARCHAR(100));")
cur.execute("INSERT INTO test_table (name) VALUES ('Test User') RETURNING id;")
conn.commit()
cur.execute("SELECT * FROM test_table;")
print("Data in test_table:", cur.fetchall())

cur.close()
conn.close()

PostgreSQL version: ('PostgreSQL 17.6 on x86_64-windows, compiled by msvc-19.44.35213, 64-bit',)
Data in test_table: [(1, 'Test User'), (2, 'Test User')]
